<a href="https://colab.research.google.com/github/VrijKun/CS6910_Assignment_1/blob/main/Assignment_1_DL_ED23D015.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install --upgrade wandb

In [5]:
import wandb
wandb.login() #3c21150eb43b007ee446a1ff6e87f640ec7528c4

wandb: Currently logged in as: ed23d015 (tea_vrij_ed23d015). Use `wandb login --relogin` to force relogin


True

# Que. 1 Solution:

In [6]:
from keras.datasets import fashion_mnist
import numpy as np
from sklearn.model_selection import train_test_split
from tqdm import tqdm

# Load the Fashion-MNIST dataset
(X_train, Y_train), (X_test, Y_test) = fashion_mnist.load_data()


image_classes = len(np.unique(Y_train)) #to find out number of unique images
image_labels = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot'] #taken from keras documentation

def show_images(image_classe,img_labels,X,Y):
   set_Labels=[]
   set_Images = []

   for i in range(len(X)):
     if len(set_Images)== image_classes:
       break
     if image_labels[Y[i]] not in set_Labels:

      set_Labels.append(image_labels[Y[i]])
      set_Images.append(X[i])

   run=wandb.init(project='DL_Assignment_1',entity='ed23d015',reinit=True)
   run.log({"Sample from each class":list(wandb.Image(img,caption= caption) for img,caption in zip(set_Images,set_Labels))})
   run.finish()

#Getting prints
show_images(image_classes,image_labels,X_test,Y_test)

wandb: Currently logged in as: ed23d015. Use `wandb login --relogin` to force relogin


In [51]:
#Class is working as well. KEEP IT

class NurlNtwk:
    def __init__(self, input_layer, hidden_layers, output_layer, initialization, activation, loss_function, optimizer, dropout_rate=0):
        self.input_layer = input_layer
        self.hidden_layers = hidden_layers
        self.output_layer = output_layer
        self.initialization = initialization
        self.activation = activation
        self.loss_function = loss_function
        self.optimizer = optimizer
        self.dropout_rate = dropout_rate
        print()
        self.parameters = None

    def initialize_parameters(self):
        np.random.seed(1)
        parameters = {}
        layer_dims = [self.input_layer] + self.hidden_layers + [self.output_layer]
        L = len(layer_dims)
        for l in range(1, L):
            if self.initialization == "random":
                parameters['W' + str(l)] = np.random.randn(layer_dims[l], layer_dims[l-1]) * 0.01
            elif self.initialization == "Xavier":
                parameters['W' + str(l)] = np.random.randn(layer_dims[l], layer_dims[l-1]) * np.sqrt(1 / layer_dims[l-1])
            parameters['b' + str(l)] = np.zeros((layer_dims[l], 1))
        self.parameters = parameters

    def sigmoid(self, Z):
        return 1 / (1 + np.exp(-Z))

    def relu(self, Z):
        return np.maximum(0, Z)

    def tanh(self, Z):
        return np.tanh(Z)

    def softmax(self, Z):
        exp_Z = np.exp(Z - np.max(Z, axis=0, keepdims=True))
        return exp_Z / np.sum(exp_Z, axis=0, keepdims=True)

    def forward_propagation(self, X):
        caches = []
        A = X
        L = len(self.parameters) // 2
        for l in range(1, L):
            A_prev = A
            Z = np.dot(self.parameters['W' + str(l)], A_prev) + self.parameters['b' + str(l)]
            if self.activation == "sigmoid":
                A = self.sigmoid(Z)
            elif self.activation == "relu":
                A = self.relu(Z)
            elif self.activation == "tanh":
                A = self.tanh(Z)
            cache = (A_prev, self.parameters['W' + str(l)], self.parameters['b' + str(l)], Z)
            caches.append(cache)
        ZL = np.dot(self.parameters['W' + str(L)], A) + self.parameters['b' + str(L)]
        AL = self.softmax(ZL)
        cache = (A, self.parameters['W' + str(L)], self.parameters['b' + str(L)], ZL)
        caches.append(cache)
        return AL, caches

    def compute_cost(self, AL, Y):
        m = Y.shape[1]
        if self.loss_function == "cross_entropy":
            cost = -np.sum(np.multiply(Y, np.log(AL))) / m
        if self.loss_function == "squared_error":
            # Default to mean squared error if loss function is not specified or recognized
            cost = np.sum((AL - Y) ** 2) / (2 * m)
        return cost

    def backward_propagation(self, AL, Y, caches):
        grads = {}
        L = len(caches)
        m = AL.shape[1]
        Y = Y.reshape(AL.shape)
        dZL = AL - Y
        A_prev, W, b, Z = caches[L-1]
        grads['dW' + str(L)] = np.dot(dZL, A_prev.T) / m
        grads['db' + str(L)] = np.sum(dZL, axis=1, keepdims=True) / m
        dAL = np.dot(W.T, dZL)
        for l in reversed(range(L-1)):
            A_prev, W, b, Z = caches[l]
            if self.activation == 'relu':
                dZ = dAL.copy()
                dZ[Z <= 0] = 0  # Derivative of ReLU
            elif self.activation == 'sigmoid':
                sigmoid_Z = self.sigmoid(Z)
                dZ = dAL * sigmoid_Z * (1 - sigmoid_Z)  # Derivative of sigmoid
            elif self.activation == 'tanh':
                tanh_Z = self.tanh(Z)
                dZ = dAL * (1 - tanh_Z**2)  # Derivative of tanh
            grads['dW' + str(l + 1)] = np.dot(dZ, A_prev.T) / m
            grads['db' + str(l + 1)] = np.sum(dZ, axis=1, keepdims=True) / m
            dAL = np.dot(W.T, dZ)
        return grads
    def initialize_optimizer(self):
        self.velocity = {}
        self.momentum = 0
        self.rmsprop_cache = {}
        self.adam_params = {'m': {}, 'v': {}}
        self.nadam_params = {'m': {}, 'v': {}}

    def update_parameters(self, grads, learning_rate):
        L = len(self.parameters) // 2
        for l in range(L):
            if self.optimizer == "SGD":
                self.parameters['W' + str(l+1)] -= learning_rate * grads['dW' + str(l+1)]
                self.parameters['b' + str(l+1)] -= learning_rate * grads['db' + str(l+1)]
            elif self.optimizer == "Momentum":
                v_dW = 0.9 * grads['dW' + str(l+1)]
                v_db = 0.9 * grads['db' + str(l+1)]
                self.parameters['W' + str(l+1)] -= learning_rate * v_dW
                self.parameters['b' + str(l+1)] -= learning_rate * v_db
            elif self.optimizer == "Nesterov":
                v_dW = 0.9 * grads['dW' + str(l+1)]
                v_db = 0.9 * grads['db' + str(l+1)]
                self.parameters['W' + str(l+1)] -= learning_rate * (v_dW + 0.9 * v_dW)
                self.parameters['b' + str(l+1)] -= learning_rate * (v_db + 0.9 * v_db)
            elif self.optimizer == "RMSprop":
                s_dW = 0.99 * grads['dW' + str(l+1)]**2 + (1 - 0.99) * (grads['dW' + str(l+1)]**2)
                s_db = 0.99 * grads['db' + str(l+1)]**2 + (1 - 0.99) * (grads['db' + str(l+1)]**2)
                self.parameters['W' + str(l+1)] -= learning_rate * grads['dW' + str(l+1)] / (np.sqrt(s_dW) + (1e-8))
                self.parameters['b' + str(l+1)] -= learning_rate * grads['db' + str(l+1)] / (np.sqrt(s_db) + (1e-8))
            elif self.optimizer == "Adam":
                v_dW = 0.9 * grads['dW' + str(l+1)] + (1 - 0.9) * grads['dW' + str(l+1)]
                v_db = 0.9 * grads['db' + str(l+1)] + (1 - 0.9) * grads['db' + str(l+1)]
                s_dW = 0.99 * grads['dW' + str(l+1)]**2 + (1 - 0.99) * (grads['dW' + str(l+1)]**2)
                s_db = 0.99 * grads['db' + str(l+1)]**2 + (1 - 0.99) * (grads['db' + str(l+1)]**2)
                self.parameters['W' + str(l+1)] -= learning_rate * v_dW / (np.sqrt(s_dW) + (1e-8))
                self.parameters['b' + str(l+1)] -= learning_rate * v_db / (np.sqrt(s_db) + (1e-8))
            elif self.optimizer == "Nadam":
                v_dW = 0.9 * grads['dW' + str(l+1)] + (1 - 0.9) * grads['dW' + str(l+1)]
                v_db = 0.9 * grads['db' + str(l+1)] + (1 - 0.9) * grads['db' + str(l+1)]
                s_dW = 0.99 * grads['dW' + str(l+1)]**2 + (1 - 0.99) * (grads['dW' + str(l+1)]**2)
                s_db = 0.99 * grads['db' + str(l+1)]**2 + (1 - 0.99) * (grads['db' + str(l+1)]**2)
                v_dW_corrected = v_dW / (1 - 0.9)
                v_db_corrected = v_db / (1 - 0.9)
                s_dW_corrected = s_dW / (1 - 0.99)
                s_db_corrected = s_db / (1 - 0.99)
                self.parameters['W' + str(l+1)] -= learning_rate * v_dW_corrected / (np.sqrt(s_dW_corrected) + (1e-8))
                self.parameters['b' + str(l+1)] -= learning_rate * v_db_corrected / (np.sqrt(s_db_corrected) + (1e-8))

    def train(self, X_train, Y_train, X_val, Y_val, num_epochs, learning_rate,batch_size, beta1=0.9, beta2=0.999, epsilon=1e-8, WandB=False):
        train_costs = []
        val_costs = []
        val_acc = []
        train_acc = []
        self.initialize_parameters()

        for epoch in range(int(num_epochs)):
          for batch in range(0, X_train.shape[1], batch_size):

                batch_images =  X_train[:,batch:batch+batch_size]
                batch_output =  train_labels[:,batch:batch+batch_size]
                AL, caches =  self.forward_propagation(batch_images)
                grads = self.backward_propagation(AL, batch_output, caches)

                if self.optimizer == "SGD":
                    self.update_parameters(grads, learning_rate)
                elif self.optimizer == "Momentum":
                    self.update_parameters(grads, learning_rate)
                elif self.optimizer== "Nesterov":
                    self.update_parameters(grads, learning_rate)
                elif self.optimizer == "RMSprop":
                    self.update_parameters(grads, learning_rate)
                elif self.optimizer == "Adam":
                    self.update_parameters(grads, learning_rate)
                elif self.optimizer == "Nadam":
                    self.update_parameters(grads, learning_rate)

          train_predictions = self.predict(X_train)
          val_predictions = self.predict(X_val)
          train_acc = self.accuracy(train_predictions, Y_train)
          val_acc = self.accuracy(val_predictions, Y_val)

          print(f"Epoch {epoch+1}/{num_epochs}, train_acc: {train_acc:.4f}, val_acc: {val_acc:.4f}")

          if WandB:
              wandb.log({"val_accuracy": val_acc,"accuracy": train_acc,"steps":epoch},)
        return train_acc

    def predict(self, X):
        AL, _ = self.forward_propagation(X)
        return np.argmax(AL, axis=0)

    def accuracy(self, predictions, labels):
        return np.mean(predictions == np.argmax(labels, axis=0))


# Load the Fashion-MNIST dataset

(train_ima, train_lab), (test_ima, test_lab) = fashion_mnist.load_data()


# Preprocess the data
train_images = train_ima / 255.0
test_images = test_ima / 255.0
train_images = train_images.reshape(train_images.shape[0], -1).T
test_images = test_images.reshape(test_images.shape[0], -1).T

def to_categorical(labels, num_classes):
    # Initialize an array of zeros with shape (num_classes, num_samples)
    one_hot = np.zeros((num_classes, labels.shape[0]))
    # Set the corresponding index of each sample to 1
    one_hot[labels, np.arange(labels.shape[0])] = 1
    return one_hot.T  # Transpose the array to match the desired shape

# Convert labels to one-hot encoding
train_labels = to_categorical(train_lab, num_classes=10).T
test_labels = to_categorical(test_lab, num_classes=10).T

# Split train set into train and validation
val_split = int(train_images.shape[1] * 0.1)
val_images = train_images[:, :val_split]
val_labels = train_labels[:, :val_split]
train_images = train_images[:, val_split:]
train_labels = train_labels[:, val_split:]


# To test diff hyperparameters

# Define hyperparameters
input_layer = train_images.shape[0]
#print(input_layer)
hidden_layers = [64,64,64,64,64]
output_layer = 10
initialization = "Xavier"
activation = "relu"
loss_function = "squared_error"
dropout_rate = 0.0
learning_rate = 0.0001
num_epochs = 10
optimizer = "Adam"
batch_size = 128
# Create and train the network
network = NurlNtwk(input_layer, hidden_layers, output_layer, initialization, activation, loss_function, optimizer, dropout_rate)
train_acc = network.train(train_images, train_labels, val_images, val_labels, num_epochs, batch_size =batch_size, learning_rate=learning_rate)
# Evaluate the model
train_predictions = network.predict(train_images)
val_predictions = network.predict(val_images)
train_acc = network.accuracy(train_predictions, train_labels)
val_acc = network.accuracy(val_predictions, val_labels)
print(f"Train Accuracy: {train_acc:.4f}")
print(f"Validation Accuracy: {val_acc:.4f}")


Epoch 1/10, train_acc: 0.7529, val_acc: 0.7567
Epoch 2/10, train_acc: 0.8051, val_acc: 0.8103
Epoch 3/10, train_acc: 0.8239, val_acc: 0.8213
Epoch 4/10, train_acc: 0.8337, val_acc: 0.8317
Epoch 5/10, train_acc: 0.8403, val_acc: 0.8383
Epoch 6/10, train_acc: 0.8456, val_acc: 0.8447
Epoch 7/10, train_acc: 0.8499, val_acc: 0.8498
Epoch 8/10, train_acc: 0.8529, val_acc: 0.8528
Epoch 9/10, train_acc: 0.8564, val_acc: 0.8570
Epoch 10/10, train_acc: 0.8585, val_acc: 0.8575
Train Accuracy: 0.8585
Validation Accuracy: 0.8575


# Que. 2 Solution:

In [22]:
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

# Preprocess the data
train_images = train_images / 255.0
test_images = test_images / 255.0
train_images = train_images.reshape(train_images.shape[0], -1).T
test_images = test_images.reshape(test_images.shape[0], -1).T


input_layer = train_images.shape[0]

hidden_layers = [64,64,64,64,64]
output_layer = 10
initialization = "Xavier"
activation = "relu"
loss_function = "cross_entropy"
dropout_rate = 0.0
learning_rate = 0.0001
num_epochs = 10
optimizer = "Adam"
batch_size = 128


# Create and train the network

network = NurlNtwk(input_layer, hidden_layers, output_layer, initialization, activation, loss_function, optimizer, dropout_rate)
network.initialize_parameters()
ProbDist,_ = network.forward_propagation(train_images)
ProbDist_t = ProbDist.T
print("Probability distribution for predicted image:",ProbDist_t[0])
print('Image class after forward propagation:' , np.argmax(ProbDist_t[0]) )
print("Actual image class(one hot encoded):",(train_labels.T)[0])



Probability distribution for predicted image: [0.09619584 0.09081578 0.08530004 0.11968298 0.10497605 0.08840017
 0.09931316 0.11695959 0.09439309 0.10396331]
Image class after forward propagation: 3
Actual image class(one hot encoded): 9


# Sweep Iteration 1

In [ ]:
sweep_config = {
    'method': 'bayes', #grid, random,bayes
    'metric': {
      'name': 'val_accuracy',
      'goal': 'maximize'
    },
    'parameters': {
        'num_epochs': {
            'values': [10,20,30]
        },
        'learning_rate': {
            'values': [0.001,0.0001]
        },
        'loss_function':{
            'values':['cross_entropy']
        },
        'initialisation':{
            'values':['random','Xavier']
        },
        'batch_size':{
            'values':[32,64,128]
        },
        'optimiser': {
            'values': ['SGD','Momentum','Nesterov', 'RMSprop','Adam','Nadam']
        },
        'activation': {
            'values': ['relu']
        },
        'hidden_layer': {
            'values': [
                       [64,64,64],[128,128,128],[256,256,256],
                       [64,64,64,64],[64,64,64,64,64],
                       [128,128,128,128],[128,128,128,128,128]]
        },

        'weight_decay':{
            'values':[0.0]
        },
        'dropout_rate':{
          'values':[0]
        }

    }
}


sweep_id = wandb.sweep(sweep_config, entity='ed23d015', project="DL_Assignment_1")

def train1():
  steps = 10
  # Default values for hyper-parameters we're going to sweep over
  config_defaults = {
      'num_epochs': 5,
      'input_layer': 784,
      'output_layer': 10,
      'batch_size':64,
      'dropout_rate':0,
      'weight_decay':0.005,
      'learning_rate': 0.0001,
      'hidden_layer':[16,16,16],
      'optimiser':'SGD',
      'activation':'sigmoid',
      'initialisation':'Xavier',
      'loss_function':'cross_entropy'

  }

  # Initialize a new wandb run
  wandb.init(project='DL_Assignment_1', entity='ed23d015',config=config_defaults)
  wandb.run.name = 'b_s:'+str(wandb.config.batch_size)+',lr:'+ str(wandb.config.learning_rate)+'drop:'+str(wandb.config.dropout_rate)+',ep:'+str(wandb.config.num_epochs)+ ',opt:'+str(wandb.config.optimiser)+ ',hl:'+str(wandb.config.hidden_layer)+ ',act:'+str(wandb.config.activation)+',decay:'+str(wandb.config.weight_decay)+',init:'+str(wandb.config.initialisation)+',loss:'+str(wandb.config.loss_function)


  config = wandb.config
  learning_rate = config.learning_rate
  num_epochs = int(config.num_epochs)
  hidden_layer = config.hidden_layer
  activation = config.activation
  optimiser = config.optimiser
  input_layer = config.input_layer
  output_layer = config.output_layer
  batch_size = config.batch_size
  weight_decay = config.weight_decay
  loss_function = config.loss_function
  initialisation = config.initialisation
  dropout_rate=config.dropout_rate
  # Training
  S_network    = NurlNtwk(input_layer, hidden_layer, output_layer,initialization=initialisation,activation=activation,loss_function=loss_function,optimizer=optimiser,dropout_rate=dropout_rate)
  acc1  = S_network.train(train_images, train_labels, val_images, val_labels, num_epochs=num_epochs, batch_size = batch_size, learning_rate=learning_rate, WandB=True)

import warnings
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning)
wandb.agent(sweep_id, function=train1)


Create sweep with ID: kht0ftnm
Sweep URL: https://wandb.ai/ed23d015/DL_Assignment_1/sweeps/kht0ftnm


wandb: Agent Starting Run: 3iu1r8fs with config:
wandb: 	activation: relu
wandb: 	batch_size: 32
wandb: 	dropout_rate: 0
wandb: 	hidden_layer: [64, 64, 64, 64, 64]
wandb: 	initialisation: Xavier
wandb: 	learning_rate: 0.0001
wandb: 	loss_function: cross_entropy
wandb: 	num_epochs: 20
wandb: 	optimiser: SGD
wandb: 	weight_decay: 0
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.



Epoch 1/20, train_acc: 0.1632, val_acc: 0.1683
Epoch 2/20, train_acc: 0.1865, val_acc: 0.1947
Epoch 3/20, train_acc: 0.2124, val_acc: 0.2195
Epoch 4/20, train_acc: 0.2287, val_acc: 0.2378
Epoch 5/20, train_acc: 0.2456, val_acc: 0.2542
Epoch 6/20, train_acc: 0.2581, val_acc: 0.2638
Epoch 7/20, train_acc: 0.2626, val_acc: 0.2663
Epoch 8/20, train_acc: 0.2656, val_acc: 0.2690
Epoch 9/20, train_acc: 0.2756, val_acc: 0.2787
Epoch 10/20, train_acc: 0.2982, val_acc: 0.2993
Epoch 11/20, train_acc: 0.3248, val_acc: 0.3288
Epoch 12/20, train_acc: 0.3684, val_acc: 0.3738
Epoch 13/20, train_acc: 0.4446, val_acc: 0.4480
Epoch 14/20, train_acc: 0.4884, val_acc: 0.4917
Epoch 15/20, train_acc: 0.5465, val_acc: 0.5488
Epoch 16/20, train_acc: 0.5878, val_acc: 0.5953
Epoch 17/20, train_acc: 0.6034, val_acc: 0.6133
Epoch 18/20, train_acc: 0.6096, val_acc: 0.6160
Epoch 19/20, train_acc: 0.6143, val_acc: 0.6172
Epoch 20/20, train_acc: 0.6188, val_acc: 0.6220


accuracy,▁▁▂▂▂▂▃▃▃▃▃▄▅▆▇█████
steps,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
val_accuracy,▁▁▂▂▂▂▃▃▃▃▃▄▅▆▇█████
accuracy,0.61876
steps,19
val_accuracy,0.622


wandb: Agent Starting Run: 2vv29161 with config:
wandb: 	activation: relu
wandb: 	batch_size: 128
wandb: 	dropout_rate: 0
wandb: 	hidden_layer: [64, 64, 64]
wandb: 	initialisation: random
wandb: 	learning_rate: 0.001
wandb: 	loss_function: cross_entropy
wandb: 	num_epochs: 20
wandb: 	optimiser: Adam
wandb: 	weight_decay: 0
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


...

wandb: Agent Starting Run: zyup1o6y with config:
wandb: 	activation: relu
wandb: 	batch_size: 128
wandb: 	dropout_rate: 0
wandb: 	hidden_layer: [128, 128, 128, 128, 128]
wandb: 	initialisation: Xavier
wandb: 	learning_rate: 0.0001
wandb: 	loss_function: cross_entropy
wandb: 	num_epochs: 20
wandb: 	optimiser: SGD
wandb: 	weight_decay: 0
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.



Epoch 1/20, train_acc: 0.1009, val_acc: 0.0937
Epoch 2/20, train_acc: 0.1012, val_acc: 0.0942
Epoch 3/20, train_acc: 0.1021, val_acc: 0.0947
Epoch 4/20, train_acc: 0.1040, val_acc: 0.0980
Epoch 5/20, train_acc: 0.1090, val_acc: 0.1015
Epoch 6/20, train_acc: 0.1180, val_acc: 0.1125
Epoch 7/20, train_acc: 0.1307, val_acc: 0.1240
Epoch 8/20, train_acc: 0.1451, val_acc: 0.1375
Epoch 9/20, train_acc: 0.1618, val_acc: 0.1523
Epoch 10/20, train_acc: 0.1784, val_acc: 0.1677
Epoch 11/20, train_acc: 0.1939, val_acc: 0.1833
Epoch 12/20, train_acc: 0.2097, val_acc: 0.1980
Epoch 13/20, train_acc: 0.2229, val_acc: 0.2143
Epoch 14/20, train_acc: 0.2336, val_acc: 0.2265
Epoch 15/20, train_acc: 0.2422, val_acc: 0.2345
Epoch 16/20, train_acc: 0.2500, val_acc: 0.2423
Epoch 17/20, train_acc: 0.2562, val_acc: 0.2482
Epoch 18/20, train_acc: 0.2613, val_acc: 0.2533
Epoch 19/20, train_acc: 0.2657, val_acc: 0.2585
Epoch 20/20, train_acc: 0.2695, val_acc: 0.2628


accuracy,▁▁▁▁▁▂▂▃▄▄▅▆▆▇▇▇▇███
steps,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
val_accuracy,▁▁▁▁▁▂▂▃▃▄▅▅▆▆▇▇▇███
accuracy,0.26952
steps,19
val_accuracy,0.26283


wandb: Agent Starting Run: ul97es4w with config:
wandb: 	activation: relu
wandb: 	batch_size: 32
wandb: 	dropout_rate: 0
wandb: 	hidden_layer: [64, 64, 64]
wandb: 	initialisation: Xavier
wandb: 	learning_rate: 0.0001
wandb: 	loss_function: cross_entropy
wandb: 	num_epochs: 30
wandb: 	optimiser: RMSprop
wandb: 	weight_decay: 0
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.



Epoch 1/30, train_acc: 0.8047, val_acc: 0.8097
Epoch 2/30, train_acc: 0.8286, val_acc: 0.8300
Epoch 3/30, train_acc: 0.8345, val_acc: 0.8347
Epoch 4/30, train_acc: 0.8388, val_acc: 0.8365
Epoch 5/30, train_acc: 0.8426, val_acc: 0.8412
Epoch 6/30, train_acc: 0.8461, val_acc: 0.8440
Epoch 7/30, train_acc: 0.8472, val_acc: 0.8492
Epoch 8/30, train_acc: 0.8469, val_acc: 0.8477
Epoch 9/30, train_acc: 0.8458, val_acc: 0.8458
Epoch 10/30, train_acc: 0.8446, val_acc: 0.8458
Epoch 11/30, train_acc: 0.8439, val_acc: 0.8437
Epoch 12/30, train_acc: 0.8463, val_acc: 0.8475
Epoch 13/30, train_acc: 0.8478, val_acc: 0.8452
Epoch 14/30, train_acc: 0.8500, val_acc: 0.8447
Epoch 15/30, train_acc: 0.8496, val_acc: 0.8493
Epoch 16/30, train_acc: 0.8484, val_acc: 0.8450
Epoch 17/30, train_acc: 0.8468, val_acc: 0.8418
Epoch 18/30, train_acc: 0.8464, val_acc: 0.8452


# Sweep Iteration 2
for Squared error loss function for hyperparameters of the highest validation accuracy


In [8]:
sweep_config = {
    'method': 'bayes', #grid, random,bayes
    'metric': {
      'name': 'val_accuracy',
      'goal': 'maximize'
    },
    'parameters': {
        'num_epochs': {
            'values': [10]
        },
        'learning_rate': {
            'values': [0.001]
        },
        'loss_function':{
            'values':['squared_error']
        },
        'initialisation':{
            'values':['Xavier']
        },
        'batch_size':{
            'values':[128]
        },
        'optimiser': {
            'values': ['Nadam']
        },
        'activation': {
            'values': ['sigmoid']
        },
        'hidden_layer': {
            'values': [
                       [64,64,64,64,64]]
        },

        'weight_decay':{
            'values':[0.0005]
        },
        'dropout_rate':{
          'values':[0]
        }

    }
}


sweep_id = wandb.sweep(sweep_config, entity='ed23d015', project="DL_Assignment_1")

def train1():
  steps = 10
  # Default values for hyper-parameters we're going to sweep over
  config_defaults = {
      'num_epochs': 5,
      'input_layer': 784,
      'output_layer': 10,
      'batch_size':64,
      'dropout_rate':0,
      'weight_decay':0.005,
      'learning_rate': 0.0001,
      'hidden_layer':[16,16,16],
      'optimiser':'SGD',
      'activation':'sigmoid',
      'initialisation':'Xavier',
      'loss_function':'squared_error'

  }

  # Initialize a new wandb run
  wandb.init(project='DL_Assignment_1', entity='ed23d015',config=config_defaults)
  wandb.run.name = 'b_s:'+str(wandb.config.batch_size)+',lr:'+ str(wandb.config.learning_rate)+'drop:'+str(wandb.config.dropout_rate)+',ep:'+str(wandb.config.num_epochs)+ ',opt:'+str(wandb.config.optimiser)+ ',hl:'+str(wandb.config.hidden_layer)+ ',act:'+str(wandb.config.activation)+',decay:'+str(wandb.config.weight_decay)+',init:'+str(wandb.config.initialisation)+',loss:'+str(wandb.config.loss_function)


  config = wandb.config
  learning_rate = config.learning_rate
  num_epochs = int(config.num_epochs)
  hidden_layer = config.hidden_layer
  activation = config.activation
  optimiser = config.optimiser
  input_layer = config.input_layer
  output_layer = config.output_layer
  batch_size = config.batch_size
  weight_decay = config.weight_decay
  loss_function = config.loss_function
  initialisation = config.initialisation
  dropout_rate=config.dropout_rate
  # Training
  S_network    = NurlNtwk(input_layer, hidden_layer, output_layer,initialization=initialisation,activation=activation,loss_function=loss_function,optimizer=optimiser,dropout_rate=dropout_rate)
  acc1  = S_network.train(train_images, train_labels, val_images, val_labels, num_epochs=num_epochs, batch_size = batch_size, learning_rate=learning_rate, WandB=True)

import warnings
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning)
wandb.agent(sweep_id, function=train1)

# For thid combination hyperparameters cross entropy "val_accuracy" is 0.8685
# For Mean_squared_error Epoch 10/10, train_acc: 0.8056, val_acc: 0.8100
# Seems like cross entropy is better loss function then squared error

Create sweep with ID: g3gbm4n8
Sweep URL: https://wandb.ai/ed23d015/DL_Assignment_1/sweeps/g3gbm4n8


wandb: Agent Starting Run: c5g5bc47 with config:
wandb: 	activation: sigmoid
wandb: 	batch_size: 128
wandb: 	dropout_rate: 0
wandb: 	hidden_layer: [64, 64, 64, 64, 64]
wandb: 	initialisation: Xavier
wandb: 	learning_rate: 0.001
wandb: 	loss_function: squared_error
wandb: 	num_epochs: 10
wandb: 	optimiser: Nadam
wandb: 	weight_decay: 0.0005
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.



Epoch 1/10, train_acc: 0.4152, val_acc: 0.4103
Epoch 2/10, train_acc: 0.5904, val_acc: 0.5965
Epoch 3/10, train_acc: 0.6768, val_acc: 0.6832
Epoch 4/10, train_acc: 0.7418, val_acc: 0.7458
Epoch 5/10, train_acc: 0.7500, val_acc: 0.7482
Epoch 6/10, train_acc: 0.7769, val_acc: 0.7763
Epoch 7/10, train_acc: 0.7893, val_acc: 0.7905
Epoch 8/10, train_acc: 0.7940, val_acc: 0.7975
Epoch 9/10, train_acc: 0.8014, val_acc: 0.8055
Epoch 10/10, train_acc: 0.8056, val_acc: 0.8100


accuracy,▁▄▆▇▇▇████
steps,▁▂▃▃▄▅▆▆▇█
val_accuracy,▁▄▆▇▇▇████
accuracy,0.80563
steps,9
val_accuracy,0.81


wandb: Ctrl + C detected. Stopping sweep.


# **Sweep Iteration 3**
For confusion matrix for **test dataset**, for hyperparameters of the highest validation accuracy  

In [59]:

sweep_config = {
    'method': 'grid', #grid, random,bayes
    'metric': {
      'name': 'val_accuracy',
      'goal': 'maximize'
    },
    'parameters': {
        'num_epochs': {
            'values': [10]
        },
        'learning_rate': {
            'values': [0.001]
        },
        'loss_function':{
            'values':['cross_entropy']
        },
        'initialisation':{
            'values':['Xavier']
        },
        'batch_size':{
            'values':[128]
        },
        'optimiser': {
            'values': ['Nadam']
        },
        'activation': {
            'values': ['tanh']
        },
        'hidden_layer': {
            'values': [
                       [64,64,64,64,64]]
        },

        'weight_decay':{
            'values':[0.0005]
        },
        'dropout_rate':{
          'values':[0]
        }

    }
}
def get_confusion_matrix():
#change the values to Best Prediction from Above run
    config_defaults = {
      'num_epochs': 5,
      'input_layer': 784,
      'output_layer': 10,
      'batch_size':64,
      'dropout_rate':0,
      'weight_decay':0.005,
      'learning_rate': 0.0001,
      'hidden_layer':[16,16,16],
      'optimiser':'SGD',
      'activation':'sigmoid',
      'initialisation':'Xavier',
      'loss_function':'squared_error'

  }

    wandb.init(config=config_defaults, magic=True)
    wandb.run.name = 'b_s:'+str(wandb.config.batch_size )+',lr:'+ str(wandb.config.learning_rate)+'drop:'+str(wandb.config.dropout_rate)+',ep:'+str(wandb.config.num_epochs)+ ',opt:'+str(wandb.config.optimiser)+ ',hl:'+str(wandb.config.hidden_layer)+ ',act:'+str(wandb.config.activation)+',decay:'+str(wandb.config.weight_decay)+',init:'+str(wandb.config.initialisation)+',loss:'+str(wandb.config.loss_function)


    class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
                    'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

    config = wandb.config
    learning_rate = config.learning_rate
    num_epochs = int(config.num_epochs)
    hidden_layer = config.hidden_layer
    activation = config.activation
    optimiser = config.optimiser
    input_layer = config.input_layer
    output_layer = config.output_layer
    batch_size = config.batch_size
    weight_decay = config.weight_decay
    loss_function = config.loss_function
    initialisation = config.initialisation
    dropout_rate=config.dropout_rate

    # Training
    S_network = NurlNtwk(input_layer, hidden_layer, output_layer,initialization=initialisation,activation=activation,loss_function=loss_function,optimizer=optimiser,dropout_rate=dropout_rate)
    acc1  = S_network.train(train_images, train_labels, val_images, val_labels, num_epochs=num_epochs, batch_size = batch_size, learning_rate=learning_rate, WandB=True)

    Y_pred = S_network.predict(test_images)
    wandb.log({"confused_matrix" : wandb.plot.confusion_matrix(probs=None,
                        y_true=test_lab, preds=Y_pred,
                        class_names=class_names)})

sweep_id = wandb.sweep(sweep_config, entity='ed23d015', project="DL_Assignment_1")
import warnings
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning)
wandb.agent(sweep_id, function=get_confusion_matrix,count=1)

Create sweep with ID: dm62vkle
Sweep URL: https://wandb.ai/ed23d015/DL_Assignment_1/sweeps/dm62vkle


wandb: Agent Starting Run: vly6ldeg with config:
wandb: 	activation: tanh
wandb: 	batch_size: 128
wandb: 	dropout_rate: 0
wandb: 	hidden_layer: [64, 64, 64, 64, 64]
wandb: 	initialisation: Xavier
wandb: 	learning_rate: 0.001
wandb: 	loss_function: cross_entropy
wandb: 	num_epochs: 10
wandb: 	optimiser: Nadam
wandb: 	weight_decay: 0.0005



Epoch 1/10, train_acc: 0.8325, val_acc: 0.8318
Epoch 2/10, train_acc: 0.8568, val_acc: 0.8557
Epoch 3/10, train_acc: 0.8677, val_acc: 0.8595
Epoch 4/10, train_acc: 0.8741, val_acc: 0.8653
Epoch 5/10, train_acc: 0.8769, val_acc: 0.8668
Epoch 6/10, train_acc: 0.8794, val_acc: 0.8677
Epoch 7/10, train_acc: 0.8836, val_acc: 0.8698
Epoch 8/10, train_acc: 0.8874, val_acc: 0.8738
Epoch 9/10, train_acc: 0.8887, val_acc: 0.8733
Epoch 10/10, train_acc: 0.8876, val_acc: 0.8710


accuracy,▁▄▅▆▇▇▇███
steps,▁▂▃▃▄▅▆▆▇█
val_accuracy,▁▅▆▇▇▇▇███
accuracy,0.88757
steps,9
val_accuracy,0.871
